# 1. 使用 FlagEmbedding
默认情况下，`FlagModel` 在编码时将使用所有可用的 GPU。请设置 `os.environ["CUDA_VISIBLE_DEVICES"]` 以选择特定的 GPU。

还可以设置 `os.environ["CUDA_VISIBLE_DEVICES"]=""` 以使所有 GPU 不可用。

## 1.1 情形一：一般情况下

In [2]:
from FlagEmbedding import FlagModel

/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["我期待的假期是在沙滩上，听着海浪声放松", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

In [4]:
model = FlagModel("/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/BAAI/bge-large-zh-v1.5", use_fp16=True)

# model = FlagModel("BAAI/bge-large-zh-v1.5", use_fp16=True)

In [5]:
embeddings_1 = model.encode(sentences_1)

In [6]:
embeddings_1.shape

(2, 1024)

In [7]:
embeddings_2 = model.encode(sentences_2)

In [8]:
embeddings_2.shape

(2, 1024)

In [9]:
# @运算符计算两组嵌入之间的点积
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.6987 0.271 ]
 [0.3416 0.37  ]]


第一行表示sentences_1中第一个句子与sentences_2中两个句子的相似度分别为0.6987和0.2712。

第二行表示第二个句子与sentences_2中两个句子的相似度分别为0.3416和0.37。

## 1.2 情形二：query比较短，passage比较长


In [10]:
queries = ["最新的AI研究成果", "健康饮食的重要性"]
passages = ["AI技术正在不断进步，最近的研究揭示了其在医疗领域的潜在应用。", "合理的饮食习惯对维持良好的身体健康至关重要，包括足够的蔬菜和水果。"]

# encode_queries()方法为每个查询自动添加指令，从而优化查询的嵌入表示
q_embeddings = model.encode_queries(queries)

# 文档的嵌入表示则可以通过encode()或encode_corpus()方法获得，因为在这种场景下，文档不需要添加指令。
p_embeddings = model.encode(passages)

# 相似性得分
scores = q_embeddings @ p_embeddings.T

print(scores)

[[0.691  0.2598]
 [0.3079 0.6953]]


# 2. 使用 Sentence-Transformers
pip install -U sentence-transformers

## 2.1 情形一：一般情况下

In [12]:
from sentence_transformers import SentenceTransformer

sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["我期待的假期是在沙滩上，听着海浪声放松", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

model = SentenceTransformer('/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/BAAI/bge-large-zh-v1.5')

embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)
embeddings_2 = model.encode(sentences_2, normalize_embeddings=True)

similarity = embeddings_1 @ embeddings_2.T

print(similarity)

[[0.69884855 0.2711424 ]
 [0.34143457 0.37027735]]


## 2.2 情形二：query比较短，passage比较长

In [15]:
from sentence_transformers import SentenceTransformer

queries = ["最新的AI研究成果", "健康饮食的重要性"]
passages = ["AI技术正在不断进步，最近的研究揭示了其在医疗领域的潜在应用。", "合理的饮食习惯对维持良好的身体健康至关重要，包括足够的蔬菜和水果。"]

instruction = "为这个句子生成表示以用于检索相关文章："

model = SentenceTransformer('/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/BAAI/bge-large-zh-v1.5')

# normalize_embeddings=True参数，确保生成的嵌入向量是归一化的
q_embeddings = model.encode([instruction+q for q in queries], normalize_embeddings=True)

p_embeddings = model.encode(passages, normalize_embeddings=True)

scores = q_embeddings @ p_embeddings.T

print(scores)

[[0.604624   0.15020016]
 [0.25311324 0.63686144]]


# 3. 使用 Langchain

In [16]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import torch
# 参考 API
# https://api.python.langchain.com/en/v0.0.345/embeddings/langchain.embeddings.huggingface.HuggingFaceBgeEmbeddings.html

In [18]:
model_name = "/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/BAAI/bge-large-zh-v1.5"

model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}

hf = HuggingFaceBgeEmbeddings(model_name = model_name,
                              model_kwargs = model_kwargs,
                              encode_kwargs = encode_kwargs,
                              query_instruction = "为这个句子生成表示以用于检索相关文章："
                              )

In [19]:
queries = ["最新的AI研究成果", "健康饮食的重要性"]
passages = ["AI技术正在不断进步，最近的研究揭示了其在医疗领域的潜在应用。", "合理的饮食习惯对维持良好的身体健康至关重要，包括足够的蔬菜和水果。"]

In [20]:
q_embeddings = [torch.tensor(hf.embed_query(query)) for query in queries]
print(q_embeddings[0].shape)
q_embeddings = torch.stack(q_embeddings)
print(q_embeddings.shape)

torch.Size([1024])
torch.Size([2, 1024])


In [21]:
p_embeddings = torch.tensor(hf.embed_documents(passages))
p_embeddings.shape

torch.Size([2, 1024])

In [22]:
scores = q_embeddings @ p_embeddings.T

print(scores)

tensor([[0.6046, 0.1502],
        [0.2531, 0.6369]])


# 4. 使用 HuggingFace Transformers

In [23]:
from transformers import AutoTokenizer, AutoModel
import torch

In [24]:
model_name = "/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/BAAI/bge-large-zh-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [25]:
# 准备模拟数据

documents = [
    "深度学习技术在计算机视觉领域中非常重要。",
    "使用深度学习模型可以理解文档的深层语义。",
    "密集检索器的优势通过学习文档和查询的表示来提高检索的准确率。"
]

query = "密集检索的优势"

In [33]:
# 获取句向量表示

def get_embedding(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        output = model(**inputs)
    # 从模型输出中提取句子嵌入。这里，我们取输出的第一个元素（通常是最后一层的隐藏状态），
    # 并且使用CLS令牌的嵌入作为句子的嵌入表示。CLS令牌位于每个序列的开头，经常被用作句子级任务的表示。
    # print(output.keys())
    # print(output[0].shape)
    # print(output[0][:,0].shape)
    embeddings = output[0][:,0]
    
    # 对句子嵌入进行L2标准化
    normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    return normalized_embeddings

In [34]:
query_embedding = get_embedding(query)

In [28]:
query_embedding.shape

torch.Size([1, 1024])

In [35]:
doc_embeddings = torch.stack([get_embedding(doc) for doc in documents]).squeeze()

In [36]:
doc_embeddings.shape

torch.Size([3, 1024])

In [37]:
# 计算相似度

from sklearn.metrics.pairwise import cosine_similarity

simialrities = cosine_similarity(query_embedding.numpy(), doc_embeddings.numpy())

print(simialrities)

[[0.32651913 0.3553676  0.72206295]]
